In [36]:
import os
import logging
import pandas as pd
import numpy as np
from questions import answer_question
from telegram import Update
from telegram.ext import ContextTypes, CommandHandler, ApplicationBuilder, MessageHandler, filters
from openai import OpenAI
import asyncio
import nest_asyncio

In [37]:
from functions import functions, run_function
import json

In [38]:
CODE_PROMPT = """
Here are two input:output examples for code generation. Please use these and follow the styling for future requests that you think are pertinent to the request.
Make sure All HTML is generated with the JSX flavoring.
// SAMPLE 1
// A Blue Box with 3 yellow cirles inside of it that have a red outline
<div style={{   backgroundColor: 'blue',
  padding: '20px',
  display: 'flex',
  justifyContent: 'space-around',
  alignItems: 'center',
  width: '300px',
  height: '100px', }}>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
</div>
""" 

In [39]:
nest_asyncio.apply()

In [40]:
df = pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

In [41]:
openai = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
tg_bot_token = os.environ['TG_BOT_TOKEN']

In [42]:
messages = [{
    "role":"system",
    "content":"You are a helpful assistant that answers questions."
},{
    "role":"system",
    "content":CODE_PROMPT
}]

In [43]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - $(levelname)s - %(message)s',
    level=logging.INFO
)

In [44]:
async def chat(update: Update, context: ContextTypes.DEFAULT_TYPE):
  messages.append({"role": "user", "content": update.message.text})
  initial_response = openai.chat.completions.create(model="gpt-3.5-turbo",
                                                    messages=messages,
                                                    tools=functions)
  initial_response_message = initial_response.choices[0].message
  messages.append(initial_response_message)
  final_response = None
  tool_calls = initial_response_message.tool_calls
    
  if tool_calls:
    for tool_call in tool_calls:
      name = tool_call.function.name
      args = json.loads(tool_call.function.arguments)
      response = run_function(name, args)
      print(tool_calls)
      messages.append({
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": name,
          "content": str(response),
      })
      if name == 'svg_to_png_bytes':
        await context.bot.send_photo(chat_id=update.effective_chat.id,
                                     photo=response)
      # Generate the final response
      final_response = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=messages,
      )
      final_answer = final_response.choices[0].message

      # Send the final response if it exists
      if (final_answer):
        messages.append(final_answer)
        await context.bot.send_message(chat_id=update.effective_chat.id,
                                       text=final_answer.content)
      else:
        # Send an error message if something went wrong
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text='something wrong happened, please try again')
  #no functions were execute
  else:
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                   text=initial_response_message.content)

In [45]:
async def mozilla(update: Update, context: ContextTypes.DEFAULT_TYPE):
    answer = answer_question(df, question=update.message.text, debug=True)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=answer)

In [46]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                  text="I am a bot, please talk to me.")

In [47]:
# Main function to run the bot
async def main() -> None:
    application = ApplicationBuilder().token(tg_bot_token).build()

    start_handler = CommandHandler('start', start)
    chat_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), chat)
    mozilla_handler = CommandHandler('mozilla', mozilla)

    application.add_handler(start_handler)
    application.add_handler(chat_handler)
    application.add_handler(mozilla_handler)

    await application.run_polling()

In [ ]:
# Check if the script is run directly (not imported)
if __name__ == '__main__':
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if str(e) == "asyncio.run() cannot be called from a running event loop":
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())
        else:
            raise

2024-07-03 16:17:59,203 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getMe "HTTP/1.1 200 OK"
2024-07-03 16:17:59,396 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/deleteWebhook "HTTP/1.1 200 OK"
2024-07-03 16:17:59,399 - apscheduler.scheduler - $(levelname)s - Scheduler started
2024-07-03 16:17:59,400 - telegram.ext.Application - $(levelname)s - Application started
2024-07-03 16:18:09,953 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:18:20,137 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:18:23,590 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1Xe

[ChatCompletionMessageToolCall(id='call_Wnmr8XdzLvX7MiQpZ5gchaqT', function=Function(arguments='{"svg_string":"<svg xmlns=\\"http://www.w3.org/2000/svg\\" width=\\"100\\" height=\\"100\\"><rect x=\\"10\\" y=\\"10\\" width=\\"80\\" height=\\"80\\" style=\\"fill:green;\\"/><rect x=\\"10\\" y=\\"10\\" width=\\"40\\" height=\\"80\\" style=\\"fill:red;\\"/><circle cx=\\"30\\" cy=\\"50\\" r=\\"15\\" fill=\\"white\\"/><text x=\\"25\\" y=\\"55\\" font-family=\\"Arial\\" font-size=\\"16\\" fill=\\"black\\">Yes</text><circle cx=\\"70\\" cy=\\"50\\" r=\\"15\\" fill=\\"white\\"/><text x=\\"65\\" y=\\"55\\" font-family=\\"Arial\\" font-size=\\"16\\" fill=\\"black\\">No</text></svg>"}', name='svg_to_png_bytes'), type='function')]


2024-07-03 16:19:18,877 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendPhoto "HTTP/1.1 200 OK"
2024-07-03 16:19:22,188 - httpx - $(levelname)s - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-03 16:19:22,697 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendMessage "HTTP/1.1 200 OK"
2024-07-03 16:19:27,905 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:19:38,091 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:19:48,274 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-

[ChatCompletionMessageToolCall(id='call_h4Bch4oujjPkbZV5M2B6YWqX', function=Function(arguments='{"svg_string":"<svg xmlns=\\"http://www.w3.org/2000/svg\\" width=\\"200\\" height=\\"50\\"><rect x=\\"10\\" y=\\"10\\" width=\\"80\\" height=\\"30\\" rx=\\"5\\" ry=\\"5\\" style=\\"fill:green;\\"/><text x=\\"50\\" y=\\"30\\" text-anchor=\\"middle\\" alignment-baseline=\\"middle\\" fill=\\"white\\">Yes</text><rect x=\\"110\\" y=\\"10\\" width=\\"80\\" height=\\"30\\" rx=\\"5\\" ry=\\"5\\" style=\\"fill:red;\\"/><text x=\\"150\\" y=\\"30\\" text-anchor=\\"middle\\" alignment-baseline=\\"middle\\" fill=\\"white\\">No</text></svg>"}', name='svg_to_png_bytes'), type='function')]


2024-07-03 16:20:06,163 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendPhoto "HTTP/1.1 200 OK"
2024-07-03 16:20:10,883 - httpx - $(levelname)s - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-03 16:20:11,348 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendMessage "HTTP/1.1 200 OK"
2024-07-03 16:20:14,917 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:20:25,100 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:20:35,284 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-

[ChatCompletionMessageToolCall(id='call_zfSTKYeuJsOEOOXeE0rNE7Fp', function=Function(arguments='{"math_string":"import math\\nmath.sqrt(91)"}', name='python_math_execution'), type='function')]


2024-07-03 16:20:46,051 - httpx - $(levelname)s - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-03 16:20:46,912 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendMessage "HTTP/1.1 200 OK"
2024-07-03 16:20:56,781 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:20:59,543 - httpx - $(levelname)s - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-03 16:21:01,044 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/sendMessage "HTTP/1.1 200 OK"
2024-07-03 16:21:07,527 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 16:21:10,047 - httpx - $(levelname)